In [27]:
import glob
import os
import numpy as np

In [28]:
#-----------------------------------------------------------------------------
# Initialize the problem data
#-----------------------------------------------------------------------------

# Read the input data file.

def read_data(filename) :
    eva_tree, graph = [],[]
    nb_node = 0
    mode,start = 0,0
    with open(filename, 'r') as fp :
        for line in fp :
          
            if (line.find('evacuation info') != -1):
#             print('Add evacuation info')             
                mode = 1
                start = 0
            
            elif (line.find('graph') != -1):
#             print('Add graph info')             
                mode = 2
                start = 0
            
            elif (mode == 1 and start == 0) :
                start = 1
                nb_evac_node = int(line.split()[0]) 
                safe_node = int(line.split()[1])
#             print('safe node is : ',safe_node)
#             print('number of evacuation nodes : ', nb_evac_node)
                          
            elif (mode == 2 and start == 0) :
                start = 1
                nb_node = int(line.split()[0])
                n = int(line.split()[1])
#             print('number of nodes : ',nb_node)
#             print('number of edges : ',n)

            
            elif (mode == 1 and start == 1) :
                data = list(map(lambda x : int(x),line.split()))
                eva_tree.append(data)

                  
            elif (mode == 2 and start == 1) :
                data = list(map(lambda x : int(x),line.split()))
                graph.append(data)
      
            
    return eva_tree,graph,nb_node


filename = os.path.dirname(os.path.abspath('__file__')) + "/data/wildfire.data"
EVA_TREE,GRAPH,NB_NODES = read_data(filename)

In [29]:
def read_solution (filename):
    solution = []
    with open(filename,'r') as fp :
        line = fp.read().splitlines()
        test_name = line[0]
        nb_evac_node = int(line[1])
        for i in range(nb_evac_node) : 
            data = list(map(lambda x : int(x),line[2+i].split()))
            solution.append(data)
            next = 2+i
        nature = line[next+1]
        f_objectif = int(line[next+2])
        methode = line[next+3]
    return solution,f_objectif,nature

filename = os.path.dirname(os.path.abspath('__file__')) + "/data/wildfire.solution"
SOLUTION,F_OBJECTIF,NATURE = read_solution(filename)

In [30]:
NB_EVA_NODES = len(EVA_TREE)
NB_EDGES = len(GRAPH)
LIST_EVA_NODES = [item[0] for item in EVA_TREE]

print('evacuation_tree = ', EVA_TREE)
print('graph = ',GRAPH)
print('number of evacuation nodes : ',NB_EVA_NODES)
print('List of evac nodes : ',LIST_EVA_NODES)
print('number of nodes = ',NB_NODES)
print('number of edges = ',NB_EDGES)
print('-----------------------------')
print('A solution is :',SOLUTION)
print('f_objectif = ',F_OBJECTIF)
print('Nature of solution : ',NATURE)

evacuation_tree =  [[1, 48, 8, 3, 4, 5, 6], [2, 30, 5, 3, 4, 5, 6], [3, 33, 3, 2, 5, 6]]
graph =  [[0, 1, 9223372036854775807, 101, 50], [0, 2, 9223372036854775807, 101, 50], [0, 3, 9223372036854775807, 101, 50], [1, 4, 9223372036854775807, 7, 8], [2, 4, 9223372036854775807, 4, 5], [3, 5, 9223372036854775807, 6, 3], [4, 5, 9223372036854775807, 9, 10], [5, 6, 9223372036854775807, 12, 11]]
number of evacuation nodes :  3
List of evac nodes :  [1, 2, 3]
number of nodes =  6
number of edges =  8
-----------------------------
A solution is : [[1, 8, 3], [2, 5, 0], [3, 3, 0]]
f_objectif =  37
Nature of solution :  valid


In [31]:
def get_eva_node_info(node_id) :
    [eva_node_info] = [item for item in EVA_TREE if item[0]==node_id]
    nb_evacuees = eva_node_info[1]
    max_rate = eva_node_info[2]
    route_length = eva_node_info[3]
    route_list = eva_node_info[4:]
    return nb_evacuees,max_rate,route_length,route_list

get_eva_node_info(3)

(33, 3, 2, [5, 6])

In [32]:
def get_edge_info(node1,node2):
    [edge_info] = [item for item in GRAPH if (item[0]==node1) & (item[1]==node2)]
    due_date = edge_info[2]
    length = edge_info[3]
    capacity = edge_info[4]
    return due_date,length,capacity

get_edge_info(1,4)

(9223372036854775807, 7, 8)

In [33]:
def get_solution_info(node_id) : 
    [sol_info] = [item for item in SOLUTION if item[:][0] == node_id]
    eva_rate = sol_info[1]
    t_start = sol_info[2]
    return eva_rate,t_start

rate,start = get_solution_info(2)
print(rate,start)

5 0


In [34]:
def get_task(node_id,eva_rate=None) :
    tasks = []
    nb_evacuees,max_rate,route_len,route_list = get_eva_node_info(node_id)
    if eva_rate == None : 
        eva_rate = max_rate
    else :
        if (eva_rate > max_rate) :
            print("ERROR ON EVACUATION RATE !!")
#         assert(eva_rate <= max_rate)
    duration = int(nb_evacuees/eva_rate)
    demande_res = [node_id] + route_list
#     print(DEMANDE_RES)
    return duration,demande_res

print(get_task(1,8)[0])
[item for item in get_task(1,8)[1]]

6


[1, 4, 5, 6]

In [35]:
for i in range(NB_EVA_NODES) : 
    node_id = EVA_TREE[i][0]
    print('Task generated by this node : ',get_task(node_id))

Task generated by this node :  (6, [1, 4, 5, 6])
Task generated by this node :  (6, [2, 4, 5, 6])
Task generated by this node :  (11, [3, 5, 6])


In [53]:
def verify_solution() : 
    
    print('evacuation_tree = ', EVA_TREE)
    print('graph = ',GRAPH)
    print('number of evacuation nodes : ',NB_EVA_NODES)
    print('List of evac nodes : ',LIST_EVA_NODES)
    print('number of nodes = ',NB_NODES)
    print('number of edges = ',NB_EDGES)
    print('-----------------------------')
    print('A solution is :',SOLUTION)
    print('f_objectif = ',F_OBJECTIF)
    print('Nature of solution : ',NATURE)
    print('-----------------------------')
    print('CREATION TASKS & VERIFY CONSTRAINTS')
    print('-----------------------------')
    ressources = {}
    for edge in GRAPH :
        edge_cap = edge[-1]
#         print('max cap of edge [{}-{}] : {}'.format(edge[0],edge[1],edge_cap))
        ressources.setdefault('Cap of edge[{}-{}]'.format(edge[0],edge[1]),np.full(40,edge_cap))
#         print(ressources)
    
    tasks = {}
    for i in LIST_EVA_NODES : 
        nb_evacuees,max_rate,route_length,route_list = get_eva_node_info(i)
        rate,start = get_solution_info(i)
        if rate > max_rate : 
            result = False
            exit(0)
#         print('task {},rate={},start={}'.format(i,rate,start))
        duration,demande_res = get_task(i,rate)
#         print(duration,demande_res)
        current = i
        for j in demande_res : 
#             print('Evacuees from {} at node {}'.format(i,j))
            nxt = j
            if current != nxt :
                _,length,edge_cap = get_edge_info(current,nxt)
                tasks.setdefault('Evacuees from {} at edge [{}-{}]'.format(i,current,nxt), [start,start+length+duration,duration,rate])
                dispo = np.copy(np.array(ressources['Cap of edge[{}-{}]'.format(current,nxt)]))
                dispo[start:start+duration] -= rate
#                 print('dispo[{}-{}]='.format(current,nxt),dispo)
                check_dispo = [item for item in dispo if item < 0]
                if (len(check_dispo) > 0) : 
                    print('TOO MANY PERSONS AT EDGE [{}-{}] !!!'.format(current,nxt))
                    result = False
                    exit(0)
#                 assert(dispo.all() >= 0)
                else :
                    ressources['Cap of edge[{}-{}]'.format(current,nxt)] = dispo
                start += length
            current = nxt
            
    print('ressources info after evacuation = ',ressources)        
    print('tasks = ', tasks)
    print('Nb of tasks = ',len(tasks))
    print('-----------------------------')
    print('CHECK F_OBJECTIF')
    print('-----------------------------')
    

    end_time = np.max([tasks[keys][1] for keys in tasks])
    print('End time is : ',end_time)
    if (end_time == F_OBJECTIF) : 
        result = True
    else :
        result = False
    
    return result

verify_solution()

evacuation_tree =  [[1, 48, 8, 3, 4, 5, 6], [2, 30, 5, 3, 4, 5, 6], [3, 33, 3, 2, 5, 6]]
graph =  [[0, 1, 9223372036854775807, 101, 50], [0, 2, 9223372036854775807, 101, 50], [0, 3, 9223372036854775807, 101, 50], [1, 4, 9223372036854775807, 7, 8], [2, 4, 9223372036854775807, 4, 5], [3, 5, 9223372036854775807, 6, 3], [4, 5, 9223372036854775807, 9, 10], [5, 6, 9223372036854775807, 12, 11]]
number of evacuation nodes :  3
List of evac nodes :  [1, 2, 3]
number of nodes =  6
number of edges =  8
-----------------------------
A solution is : [[1, 8, 3], [2, 5, 0], [3, 3, 0]]
f_objectif =  37
Nature of solution :  valid
-----------------------------
CREATION TASKS & VERIFY CONSTRAINTS
-----------------------------
ressources info after evacuation =  {'Cap of edge[0-1]': array([50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
       50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
       50, 50, 50, 50, 50, 50]), 'Cap of edge[0-2]': array([50, 50, 50, 50,

True

In [37]:
def get_duration(node_id) :
    nb_evacuees,cap_max,route_len,route_list = get_eva_node_info(node_id)
    
    E_tmp =  nb_evacuees / cap_max
    node_current = node_id
    for i in route_list:
        node_next = i
#         [edge_info] = [item for item in GRAPH if (item[0]==node_current) & (item[1]==node_next)]
        _,length,edge_cap= get_edge_info(node_current,node_next)
        assert(cap_max <= edge_cap)
        E_tmp += length
        # reload the next edge
        node_current = node_next
    #end for
    return int(E_tmp)

get_duration(1)

34

In [38]:
for i in range(NB_EVA_NODES) : 
    node_id = EVA_TREE[i][0]
    print('Evacuation of (only) node ',node_id,' is ', get_duration(node_id),' mins')

Evacuation of (only) node  1  is  34  mins
Evacuation of (only) node  2  is  31  mins
Evacuation of (only) node  3  is  29  mins


In [39]:
def get_borne_inf() : 
    eva_time = [get_duration(item) for item in LIST_EVA_NODES]
    return np.max(eva_time)

print('Borne inferieur = {} mins'.format(get_borne_inf()))

Borne inferieur = 34 mins


In [146]:
def get_borne_sup() :
    
    ressources = {}
    for edge in GRAPH :
        edge_cap = edge[-1]
#         print('max cap of edge [{}-{}] : {}'.format(edge[0],edge[1],edge_cap))
        ressources.setdefault('Cap of edge[{}-{}]'.format(edge[0],edge[1]),np.full(40,edge_cap))
#         print(ressources)
    
    tasks = {}
    for i in LIST_EVA_NODES : 
        nb_evacuees,max_rate,route_length,route_list = get_eva_node_info(i)
        start = 0
        duration,demande_res = get_task(i,max_rate)
        current = i
         
        for j in demande_res : 
#             print('Evacuees from {} at node {}'.format(i,j))
            nxt = j
            if current != nxt :
                _,length,edge_cap = get_edge_info(current,nxt)

                ok = False
                while (not ok) :
                    dispo = np.copy(np.array(ressources['Cap of edge[{}-{}]'.format(current,nxt)]))
                    dispo[start:start+duration] -= max_rate
#                     print('dispo[{}-{}]={}'.format(current,nxt,dispo))
                    check_dispo = [item for item in dispo if item < 0]
#                     print(check_dispo)
                    if (len(check_dispo) > 0) :        
#                         print('OVERLOAD')
                        start += len(check_dispo) 
                        ok = False
                        # Change the time start of the previous nodes 
                        for keys in tasks :
                            if 'Evacuees from {}'.format(i) in keys :
                                tasks[keys][0] += len(check_dispo)
                    else : 
                        ok = True
                        ressources['Cap of edge[{}-{}]'.format(current,nxt)] = dispo

                tasks.setdefault('Evacuees from {} at edge [{}-{}]'.format(i,current,nxt), [start,start+length+duration,duration,max_rate])
        
                start += length
            current = nxt
            
#         print('ressources info after evacuation of node {} with rate{} = {}'.format(i,max_rate,ressources))
        
    print('tasks = ', tasks)
    print('Nb of tasks = ',len(tasks))
    
    end_time = np.max([tasks[keys][1] for keys in tasks])
#     print([tasks[keys][1] for keys in tasks])
    print('End time is : ',end_time)
    
    return end_time

get_borne_sup()

tasks =  {'Evacuees from 1 at edge [1-4]': [0, 13, 6, 8], 'Evacuees from 1 at edge [4-5]': [7, 22, 6, 8], 'Evacuees from 1 at edge [5-6]': [16, 34, 6, 8], 'Evacuees from 2 at edge [2-4]': [9, 10, 6, 5], 'Evacuees from 2 at edge [4-5]': [13, 28, 6, 5], 'Evacuees from 2 at edge [5-6]': [22, 40, 6, 5], 'Evacuees from 3 at edge [3-5]': [0, 17, 11, 3], 'Evacuees from 3 at edge [5-6]': [6, 29, 11, 3]}
Nb of tasks =  8
End time is :  40


40

In [106]:
a = np.array([1,2,3,4])
b = np.copy(a[:2])
b

array([1, 2])

In [107]:
b-=1
b

array([0, 1])

In [111]:
c = [0,0,0,0]
c

[0, 0, 0, 0]

In [113]:
c[:2] = b
c

[0, 1, 0, 0]